In [1]:
CUDA_VISIBLE_DEVICES=0,1
import sys
sys.path.append('/media/dataserver/workspace/blanca/utils/')

from utils import libraries
# from utils import utils_global
from utils_global import *

%matplotlib inline
import imageio

/root/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
def enumeration(): #*args):
    members = args[0] if len(args) == 1 else args
    def checker(value):
        if value.strip().lower() not in members:
            raise ValueError(value)
    return checker


enumeration()

<function __main__.enumeration.<locals>.checker>

In [3]:
def a(l, i):
    'returns a function that given "x" adds x to list l at index i'
    def aa(x): l[i] = x
    return aa

def b(x, y): 
    'given *args: it returns x(y(*args))'
    return lambda *args: x(y(*args))

def c(fn): 
    'given arg and *args: it returns fn(ar, *args) --> passes (arg, *args) to another fn'
    return lambda arg: lambda *args: fn(arg,*args)

def g(iterable, fn): 
    'aplies fn to (index, x) of a e.g. a list'
    return (fn(i, x) for i, x in enumerate(iterable))

def foo(l, f):
    a_ = c(a)(l) # instanciate aa(l,)  l as list
    g_ = c(g)(l) # instanciate g(l, )  l as iterable (list)
    bar = lambda i, x: b(a_(i), f)(x) # given i, x: aa(l, i) (f(x)) --> for i in l: l[i] = f(x) 
    for i in g_(bar): 
        # --> for ix, i in enumerate(l): bar(ix, i)   equivalent to:
        # --> for ix, i in enumerate(l): l[ix] = f(i)
        pass

# example:
ml = [1, 2, 3]
mff = lambda x, y: (y * x) ** 2 
mf = lambda x: mff(x, 10)
foo(ml, mf)

# ml --> [100, 400, 900]

In [21]:
def enumeration(*args):
    members = args[0] if len(args) == 1 else args
    def checker(value):
        if value.strip().lower() not in members:
            raise ValueError(value)
    return checker

The function takes a list and an argument x and checks if x (case-insensitive and omitting begining/ending spaces) is in the list; otherwise it raises ValueError

# example
my_members = ['a', 'b', 'c']
members_to_check_ok = ['A', '  a  ', 'B   ', '   c']
members_to_check_fail = ['A', '  a  ', 'B   ', '   D']

# this will not fail as all elements belong to to my_members (lowercased and without spaces)
for i in members_to_check_ok:
    enumeration(my_members)(i)
    
# this will raise "ValueError:    D"
for i in members_to_check_fail:
    enumeration(my_members)(i) 

In [60]:
# optimization problem 

# from surprise import Reader, Dataset
# # Define the format

# reader = Reader(line_format='user item rating timestamp', sep='\t')
# # Load the data from the file using the reader format
# data = Dataset.load_from_file('/home/ubuntu/.surprise_data/ml-100k/ml-100k/u.data', reader=reader)


file = data_path
reader = Reader(line_format='userid itemid rating', sep=',')
data = Dataset.load_from_file(file, reader=reader)

# train_file = os.path.expanduser(data_path + 'train.data')
# test_file = os.path.expanduser(data_path + 'test.data')
# reader = Reader(line_format='user item rating timestamp', sep=';')
# data = Dataset.load_from_folds([(train_file, test_file)], reader=reader)

   UserID  ItemID  Rating
0       0     194       3
1       0      96       3
2       0      34       4
3       0      42       3
4       0     675       4
(29586, 3)


ValueError: line_format parameter is incorrect.

In [70]:
import os
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader

data_path='/media/dataserver/workspace/blanca/publicis/recommender/user_item_ratings.csv'
ratings = pd.read_csv(data_path)
print(ratings.head())
print(ratings.shape)

n_users=int(ratings.UserID.nunique())
n_movies=int(ratings.ItemID.nunique())

print("n_users = ",n_users, "|", "n_movies = ", n_movies )

g = ratings.groupby('UserID')['Rating'].count()
topg = g.sort_values(ascending = False)[:15]

i = ratings.groupby('ItemID')['Rating'].count()
topi = i.sort_values(ascending = False)[:15]

   UserID  ItemID  Rating
0       0     194       3
1       0      96       3
2       0      34       4
3       0      42       3
4       0     675       4
(29586, 3)
n_users =  1912 | n_movies =  805


In [71]:
# gettings ratings of top users and top items

join1 = ratings.join(topg, on='UserID', how = 'inner', rsuffix='_r')
join1 = join1.join(topi, on='ItemID', how = 'inner', rsuffix = '_r')

pd.crosstab(join1.UserID, join1.ItemID, join1.Rating, aggfunc=np.sum)

ItemID,27,104,107,111,124,187,222,313,316,442,641
UserID,,,,,,,,,,,
199,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
204,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN
603,2.0,NaN,NaN,2.0,4.0,NaN,NaN,4.0,NaN,NaN,NaN
607,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
608,2.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
633,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1243,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0
1624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


In [77]:
#### Collaborative filtering

ratings_dict = {'itemID': list(ratings.ItemID),
                'userID': list(ratings.UserID),
                'rating': list(ratings.Rating)}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5.0))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

ratings[:2]

,UserID,ItemID,Rating
0,0,194,3
1,0,96,3


In [78]:
data.split(n_folds=5) # split data into 5 folds
from surprise import SVD, evaluate
from surprise import GridSearch

param_grid = {'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
grid_search = GridSearch(SVD, param_grid, measures=['RMSE'])
grid_search.evaluate(data)

/root/miniconda3/lib/python3.6/site-packages/surprise/evaluate.py:232: UserWarning: The GridSearch() class is deprecated. Please use model_selection.GridSearchCV instead.
  'model_selection.GridSearchCV instead.', UserWarning)
/root/miniconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/root/miniconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Running grid search for the following parameter combinations:
{'lr_all': 0.002, 'reg_all': 0.4}
{'lr_all': 0.002, 'reg_all': 0.6}
{'lr_all': 0.005, 'reg_all': 0.4}
{'lr_all': 0.005, 'reg_all': 0.6}
Resulsts:
{'lr_all': 0.002, 'reg_all': 0.4}
{'RMSE': 0.69865546677145918}
----------
{'lr_all': 0.002, 'reg_all': 0.6}
{'RMSE': 0.70300265820337737}
----------
{'lr_all': 0.005, 'reg_all': 0.4}
{'RMSE': 0.67218878423474737}
----------
{'lr_all': 0.005, 'reg_all': 0.6}
{'RMSE': 0.67948884941754228}
----------


In [79]:
algo = SVD()
evaluate(algo, data, measures=['RMSE'])

/root/miniconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/root/miniconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.6624
------------
Fold 2
RMSE: 0.6758
------------
Fold 3
RMSE: 0.6716
------------
Fold 4
RMSE: 0.6681
------------
Fold 5
RMSE: 0.6713
------------
------------
Mean RMSE: 0.6698
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.6624274154475096,
                             0.6758190430797496,
                             0.67156687164330842,
                             0.66808895936935997,
                             0.67125577921257273]})

In [80]:
from surprise import KNNBasic
algo = KNNBasic()
evaluate(algo, data, measures=['RMSE', 'MAE'])

/root/miniconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/root/miniconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7846
MAE:  0.6053
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7911
MAE:  0.6087
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7986
MAE:  0.6134
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7776
MAE:  0.6024
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7874
MAE:  0.6048
------------
------------
Mean RMSE: 0.7878
Mean MAE : 0.6069
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.60525674535696539,
                             0.60867310177835365,
                             0.61340954627042155,
                             0.60240989273060319,
                             0.60478165266536754],
                            'rmse': [0.78460543933999038,
                             0.79108063520125382,
                             0.79859617217585799,
                             0.77756108233438292,
                             0.78737116421041486]})

In [81]:
from surprise import NMF
algo = NMF()
evaluate(algo, data, measures=['RMSE'])

/root/miniconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/root/miniconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm NMF.

------------
Fold 1
RMSE: 0.7152
------------
Fold 2
RMSE: 0.7292
------------
Fold 3
RMSE: 0.7177
------------
Fold 4
RMSE: 0.7194
------------
Fold 5
RMSE: 0.7233
------------
------------
Mean RMSE: 0.7210
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.7152469073000961,
                             0.72916754485513002,
                             0.7176851952952702,
                             0.71942922170569412,
                             0.72333438225025182]})

In [84]:
#### cosine distance

ratings2 = ratings.copy()
col = ['ItemID', 'UserID']

for c in col:
    ratings2[c].replace({val: i for i, val in enumerate(ratings2[c].unique())}, inplace=True)

ratings2[:2]

n_users=int(ratings2.UserID.nunique())
n_items=int(ratings2.ItemID.nunique())

print("n_users = ",n_users, "||", "n_items = ", n_items )

from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(ratings2, test_size=0.25)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

n_users =  1912 || n_items =  805


/root/miniconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [85]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [88]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

from sklearn.metrics import mean_squared_error
from math import sqrt
def mse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return mean_squared_error(prediction, ground_truth)

print('User-based CF MSE: ' , str(mse(user_prediction, test_data_matrix)))
print('Item-based CF MSE: ' , str(mse(item_prediction, test_data_matrix)))

User-based CF MSE:  8.93311840697
Item-based CF MSE:  9.00297875215


In [93]:
# import matplotlib.pyplot as plt

# mses = [6.47, .957, .897, .804, .801, .79]
# algos = ['cosine_memory', 'KNN', "NMF", 'SVD', 'PMF', 'DL']
# plt.plot(algos, mses, 'go',  )
# plt.xlabel("Different algos")
# plt.ylabel("MSE")
# plt.show()

# FileLink('collaborating_filter.ipynb')

In [94]:
# with Spotlight
from spotlight.interactions import Interactions
interactions = Interactions(np.asarray(userid, dtype=np.int32),
                            np.asarray(bookid, dtype=np.int32),
                            np.asarray(ratings_all, dtype=np.float32))

ModuleNotFoundError: No module named 'spotlight'

In [95]:
#### optimization

In [96]:
from scipy.optimize import minimize
import numpy as np

def f(coord,x,y,r):
    return np.sum( ((coord[0] - x)**2) + ((coord[1] - y)**2) - (r**2) )

x = np.array([0,   2,  0])
y = np.array([0,   0,  2])
r = np.array([.88, 1, .75])

# initial (bad) guess at (x,y) values
initial_guess = np.array([100,100])

res = minimize(f,initial_guess,args = [x,y,r])

TypeError: f() missing 2 required positional arguments: 'y' and 'r'